In [21]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge-score py7zr -q


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
%pip install --upgrade acceletate
%pip install -y transformers accelerate
%pip install transformers accelerate

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement acceletate (from versions: none)
ERROR: No matching distribution found for acceletate

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\siddh\OneDrive\Desktop\Projects\Text_Summarizer\myenv\Scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\siddh\OneDrive\Desktop\Projects\Text_Summarizer\myenv\Scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\siddh\OneDrive\Desktop\Projects\Text_Summarizer\myenv\Scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\siddh\OneDrive\Desktop\Projects\Text_Summarizer\myenv\Scripts\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\siddh\OneDrive\Desktop\Projects\Text_Summarizer\myenv\Scripts\python.exe -m pip install [options] <archive url/path> ...

no such option: -y


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset , load_metric
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [25]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
#dowload & unzip data

!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [26]:
import pandas as pd
import json
import os

# List of JSON files
json_files = ['train.json', 'test.json', 'val.json']

# Iterate through each JSON file
for json_file in json_files:
    # Construct the full path to the JSON file
    json_file_path = os.path.join('dataset', json_file)

    # Load JSON file
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Convert to DataFrame
    df = pd.json_normalize(data)

    # Save to CSV
    csv_output_path = os.path.join('dataset', f'{json_file.split(".")[0]}.csv')
    df.to_csv(csv_output_path, index=False)
    print(f'CSV file saved to: {csv_output_path}')


CSV file saved to: dataset\train.csv
CSV file saved to: dataset\test.csv
CSV file saved to: dataset\val.csv


In [28]:
train_path = 'dataset/train.csv'
training_dataset = pd.read_csv(train_path)
training_dataset

,id,summary,dialogue
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella...."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...
...,...,...,...
14727,13863028,Romeo is trying to get Greta to add him to her...,Romeo: You are on my ‘People you may know’ lis...
14728,13828570,Theresa is at work. She gets free food and fre...,Theresa: <file_photo>\r\nTheresa: <file_photo>...
14729,13819050,Japan is going to hunt whales again. Island an...,John: Every day some bad news. Japan will hunt...
14730,13828395,Celia couldn't make it to the afternoon with t...,Jennifer: Dear Celia! How are you doing?\r\nJe...


In [29]:
print(f"Features: {training_dataset.columns}")

Features: Index(['id', 'summary', 'dialogue'], dtype='object')


In [32]:
def convert_examples_to_features(example_batch):
   input_encodings = tokenizer(example_batch['dialogue'].tolist(), max_length= 1024, truncation= True )

   with tokenizer.as_target_tokenizer():
      target_encodings = tokenizer(example_batch['dialogue'].tolist(), max_length= 128, truncation= True )

   return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'target_ids': target_encodings['input_ids'],
   }

In [35]:
dataset_samsum= load_from_disk('dataset')

FileNotFoundError: Directory dataset is neither a `Dataset` directory nor a `DatasetDict` directory.

In [34]:
training_dataset_pt = training_dataset.map(convert_examples_to_features)

TypeError: string indices must be integers